In [8]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split, Dataset
import random
import itertools

In [9]:
name = 'FD001'
seq_len = 30
Rc = 130

In [30]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')


NVIDIA RTX A6000


# Data Loading



In [21]:
class load_data(Dataset):
    """
    root = new | old
    """
    def __init__(self, name, seq_len, root='new') -> None:
        super().__init__()
        data_root = "data/units/"
        if root == 'old':
            label_root = "data/labels/"
        elif root == 'new':
            label_root = "data/new_labels/"
        else:
            raise RuntimeError("got invalid parameter root='{}'".format(root))
        raw = np.loadtxt(data_root+name)[:,2:]
        lbl = np.loadtxt(label_root+name)/Rc
        l = len(lbl)
        if l<seq_len:
            raise RuntimeError("seq_len {} is too big for file '{}' with length {}".format(seq_len, name, l))
        raw, lbl = torch.tensor(raw, dtype=torch.float), torch.tensor(lbl, dtype=torch.float)
        lbl_pad_0 = [torch.ones([seq_len-i-1]) for i in range(seq_len-1)] 
        data_pad_0 = [torch.zeros([seq_len-i-1,24]) for i in range(seq_len-1)]
        lbl_pad_1 = [torch.zeros([i+1]) for i in range(seq_len-1)] 
        data_pad_1 = [torch.zeros([i+1,24]) for i in range(seq_len-1)]
        self.data = [torch.cat([data_pad_0[i],raw[:i+1]],0) for i in range(seq_len-1)] 
        self.data += [raw[i-seq_len+1:i+1] for i in range(seq_len-1, l)]
        self.data += [torch.cat([raw[l-seq_len+i+1:], data_pad_1[i]],0) for i in range(seq_len-1)]
        self.label = [torch.cat([lbl_pad_0[i],lbl[:i+1]],0) for i in range(seq_len-1)] 
        self.label += [lbl[i-seq_len+1:i+1] for i in range(seq_len-1, l)]
        self.label += [torch.cat([lbl[l-seq_len+i+1:], lbl_pad_1[i]],0) for i in range(seq_len-1)]
        self.padding = [torch.cat([torch.ones(seq_len-i-1), torch.zeros(i+1)],0) for i in range(seq_len-1)]   # 1 for ingore
        self.padding += [torch.zeros(seq_len) for i in range(seq_len-1, l)]
        self.padding += [torch.cat([torch.zeros(seq_len-i-1), torch.ones(i+1)],0) for i in range(seq_len-1)]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.label[index], self.padding[index]


class load_all_data(Dataset):
    """
    root: new | old
    name: LIST of txt files to collect 
    """
    def __init__(self, name, seq_len) -> None:
        super().__init__()
        data_root = "data/units/"
        label_root = "data/new_labels/"
        lis = os.listdir(data_root)
        data_list = [i for i in lis if i in name]
        self.data, self.label, self.padding = [], [], []
        for n in data_list:
            raw = np.loadtxt(data_root+n)[:,2:]
            lbl = np.loadtxt(label_root+n)/Rc
            l = len(lbl)
            if l<seq_len:
                raise RuntimeError("seq_len {} is too big for file '{}' with length {}".format(seq_len, n, l))
            raw, lbl = torch.tensor(raw, dtype=torch.float), torch.tensor(lbl, dtype=torch.float)
            lbl_pad_0 = [torch.ones([seq_len-i-1]) for i in range(seq_len-1)] 
            data_pad_0 = [torch.zeros([seq_len-i-1,24]) for i in range(seq_len-1)]
            lbl_pad_1 = [torch.zeros([i+1]) for i in range(seq_len-1)] 
            data_pad_1 = [torch.zeros([i+1,24]) for i in range(seq_len-1)]
            self.data += [torch.cat([data_pad_0[i],raw[:i+1]],0) for i in range(seq_len-1)] 
            self.data += [raw[i-seq_len+1:i+1] for i in range(seq_len-1, l)]
            self.data += [torch.cat([raw[l-seq_len+i+1:], data_pad_1[i]],0) for i in range(seq_len-1)]
            self.label += [torch.cat([lbl_pad_0[i],lbl[:i+1]],0) for i in range(seq_len-1)] 
            self.label += [lbl[i-seq_len+1:i+1] for i in range(seq_len-1, l)]
            self.label += [torch.cat([lbl[l-seq_len+i+1:], lbl_pad_1[i]],0) for i in range(seq_len-1)]
            self.padding += [torch.cat([torch.ones(seq_len-i-1), torch.zeros(i+1)],0) for i in range(seq_len-1)]   # 1 for ingore
            self.padding += [torch.zeros(seq_len) for i in range(seq_len-1, l)]
            self.padding += [torch.cat([torch.zeros(seq_len-i-1), torch.ones(i+1)],0) for i in range(seq_len-1)]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.label[index], self.padding[index]

In [22]:
tr = np.loadtxt("save/"+name+"/train"+name+".txt", dtype=str).tolist()
val = np.loadtxt("save/"+name+"/valid"+name+".txt", dtype=str).tolist()
ts = np.loadtxt("save/"+name+"/test"+name+".txt", dtype=str).tolist()





# Model

In [53]:
from models import *
model = transformer(max_len=seq_len)
model.to(device)
Loss = nn.MSELoss()
Loss.to(device)
opt = torch.optim.Adam(model.parameters(), lr=0.02)



In [55]:
def train(data, model, Loss, opt, seq_len):
    min = 999
    for e in range(10):
        model.train()
        random.shuffle(data)
        train_data = load_all_data(data, seq_len=seq_len)
        total_loss = 0
        train_iter = iter(DataLoader(train_data, batch_size=32, shuffle=True))
        counter = 0
        for _ in range(len(train_iter)):
            train_data, train_label, train_padding = next(train_iter)
            train_data, train_label, train_padding = train_data.to(device), train_label.to(device), train_padding.to(device)
            feature, out = model(train_data, train_padding)
            out.squeeze_(2)
            loss = Loss(out, train_label)
            total_loss += loss
            counter += 1
            opt.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 2)
            opt.step()
        rmse = validate()
        print("Epoch: {}, Loss: {}, RMSE: {}".format(e, total_loss/counter, rmse))
        if (rmse < min):
            min = rmse
            torch.save(model.state_dict(), 'save/transformer.pth')
    return min
    
            
            
            
        
def validate():
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for i in val:
            pred_sum, pred_cnt = torch.zeros(800), torch.zeros(800)
            pred_sum.cpu()
            valid_data = load_data(i, seq_len)
            data_len = len(valid_data)
            
            valid_loader = DataLoader(valid_data, batch_size=1000)
            valid_iter = iter(valid_loader)
           
            d = next(valid_iter)
            
            valid_data, valid_label, valid_padding = d[0].to(device), d[1], d[2].to(device)
            _, out = model(valid_data, valid_padding)
            out = out.squeeze(2).cpu()
            
            for j in range(data_len):
                if j < seq_len-1:
                   
                    pred_sum[:j+1] += out[j, -(j+1):]
                    pred_cnt[:j+1] += 1
                elif j <= data_len-seq_len:
                    pred_sum[j-seq_len+1:j+1] += out[j]
                    pred_cnt[j-seq_len+1:j+1] += 1
                else:
                    pred_sum[data_len-seq_len+1-(data_len-j):data_len-seq_len+1] += out[j, :(data_len-j)]
                    pred_cnt[data_len-seq_len+1-(data_len-j):data_len-seq_len+1] += 1
            truth = torch.tensor([valid_label[j,-1] for j in range(len(valid_label)-seq_len+1)], dtype=torch.float)
            pred_sum, pred_cnt = pred_sum[:data_len-seq_len+1], pred_cnt[:data_len-seq_len+1]
            pred = pred_sum/pred_cnt
            mse = float(torch.sum(torch.pow(pred-truth, 2)))
            rmse = math.sqrt(mse/data_len)
            total_loss += rmse
    return total_loss*Rc/len(val)
train(tr, model, Loss, opt, seq_len)

Epoch: 0, Loss: 0.1389843374490738, RMSE: 41.24155383412785
Epoch: 1, Loss: 0.039381109178066254, RMSE: 13.512601583874106
Epoch: 2, Loss: 0.02773537114262581, RMSE: 14.16272218453192
Epoch: 3, Loss: 0.024204375222325325, RMSE: 27.801526867864215
Epoch: 4, Loss: 0.023299481719732285, RMSE: 18.009024196467042
Epoch: 5, Loss: 0.022514406591653824, RMSE: 16.056866869046445
Epoch: 6, Loss: 0.024206873029470444, RMSE: 16.46752404569999
Epoch: 7, Loss: 0.025684958323836327, RMSE: 23.039852733566047
Epoch: 8, Loss: 0.022637737914919853, RMSE: 17.640329246170786
Epoch: 9, Loss: 0.02205568738281727, RMSE: 16.991854282436154


13.512601583874106

In [56]:
def score(pred, truth):
    """input must be tensors!"""
    x = pred-truth
    score1 = torch.tensor([torch.exp(-i/13)-1 for i in x if i<0])
    score2 = torch.tensor([torch.exp(i/10)-1 for i in x if i>=0])
    return int(torch.sum(score1)+torch.sum(score2))


def get_pred_result(data_len, out, lb):
    pred_sum, pred_cnt = torch.zeros(800), torch.zeros(800)
    for j in range(data_len):
        if j < seq_len-1:
            pred_sum[:j+1] += out[j, -(j+1):]
            pred_cnt[:j+1] += 1
        elif j <= data_len-seq_len:
            pred_sum[j-seq_len+1:j+1] += out[j]
            pred_cnt[j-seq_len+1:j+1] += 1
        else:
            pred_sum[data_len-seq_len+1-(data_len-j):data_len-seq_len+1] += out[j, :(data_len-j)]
            pred_cnt[data_len-seq_len+1-(data_len-j):data_len-seq_len+1] += 1
    truth = torch.tensor([lb[j,-1] for j in range(len(lb)-seq_len+1)], dtype=torch.float)
    pred_sum, pred_cnt = pred_sum[:data_len-seq_len+1], pred_cnt[:data_len-seq_len+1]
    pred2 = pred_sum/pred_cnt
    pred2 *= Rc
    truth *= Rc
    return truth, pred2 


In [ ]:
def test():
    truth, tot, tot_sc = [], 0, 0
    model.eval()
    
    with torch.no_grad():
        for k in range(test_len):
            i = next(list_iter)
            dataset = TRANSFORMERDATA(i, seq_len)
            data_len = len(dataset)
            dataloader = DataLoader(dataset, batch_size=800, shuffle=0)
            it = iter(dataloader)
            d = next(it)
            input, lb, msk = d[0], d[1], d[2]
            if fake:
                input = torch.zeros(input.shape)
            input, msk = input.cuda(), msk.cuda()
            #uncertainty(input, msk, data_len, lb, i)
            _, out = net(input, msk)
            out = out.squeeze(2).cpu()
            truth, pred = get_pred_result(data_len, out, lb)
            mse = float(torch.sum(torch.pow(pred-truth, 2)))
            rmse = math.sqrt(mse/data_len)
            tot += rmse
            sc = score(pred, truth)
            tot_sc += sc
            print("for file {}: rmse={:.4f}, score={}".format(i, rmse, sc))
            print('-'*80)
           
    print("tested on [{}] files, mean RMSE = {:.4f}, mean score = {}".format(test_len, tot/test_len, int(tot_sc/test_len)))


In [ ]:
x=torch.load("save/transformer.pth", map_location='cuda:0')
model.load_state_dict(x)
data_root = "data/units/"
label_root = "data/labels/"
lis = os.listdir(data_root)
test_list = [i for i in lis if i[:5] == name]
random.shuffle(test_list)
test_len = len(test_list)
list_iter = iter(test_list)
test()
    